# SVM in ORC(Optical Character Recognition)
## 手写数字图片的OCR

在kNN中，我们直接使用**像素强度**作为特征向量。这次我们将使用**定向梯度直方图（HOG）**作为特征向量。

## 偏斜校准
1. mu02：关于 y 轴的对称性矩。它度量了图像在 y 轴方向上的分布特征。具体而言，它是通过将图像中每个像素的 y 坐标值与图像的中心 y 坐标值进行差值，然后将差值平方后求和得到的。

2. mu11：关于 x 轴和 y 轴的对称性矩。它度量了图像在 x 轴和 y 轴方向上的相关性特征。具体而言，它是通过将图像中每个像素的 x 坐标值与图像的中心 x 坐标值相乘，再将乘积与每个像素的 y 坐标值相乘，最后将乘积求和得到的。

当将 mu11 除以 mu02 时，可以得到一个值，代表了图像在 x 轴和 y 轴方向上的相关性特征与关于 y 轴的对称性特征之间的比例关系。

这个比值通常被称为**偏斜度**（skewness）

In [21]:
#!/usr/bin/env python
import cv2 as cv
import numpy as np

SZ = 20     # 训练图像的大小
bin_n = 16  # 柱状图的箱子数
affine_flags = cv.WARP_INVERSE_MAP | cv.INTER_LINEAR



#数字图像并对其进行偏斜校正
def deskew(img):
    # 计算二阶矩矩阵mu02和mu11
    # mu02是关于y轴的对称性，mu11是关于x轴和y轴的对称性
    m = cv.moments(img)
    # 如果二阶矩矩阵mu02的绝对值小于1e-2，那么就不需要进行偏斜校正
    if abs(m['mu02']) < 1e-2:#1e-2=0.01 科学计数法
        return img.copy()
    # 计算偏斜校正的角度
    skew = m['mu11'] / m['mu02']
    # 计算仿射变换矩阵
    #   [[ 1.         -0.34675327  3.4675326 ]
    #   [ 0.          1.          0.        ]]
    M = np.float32([[1, skew, -0.5 * SZ * skew], [0, 1, 0]])
    # 进行仿射变换
    img = cv.warpAffine(img, M, (SZ, SZ), flags=affine_flags)
    return img

##HOG描述符是一种特征描述符，它可以用来描述图像的形状和轮廓。
# HOG描述符的计算步骤如下：
# 1.计算图像的梯度，得到图像的梯度幅值和梯度方向；
# 2.将图像分割成若干个小的细胞单元，每个细胞单元包含若干个像素；
# 3.对每个细胞单元内的像素，根据梯度方向将其投影到梯度方向上，得到每个细胞单元的梯度直方图；
# 4.将每个细胞单元的梯度直方图进行归一化，得到每个细胞单元的HOG特征向量；
# 5.将每个细胞单元的HOG特征向量进行连接，得到整幅图像的HOG特征向量。
def hog(img):
    # 计算图像的梯度
    gx = cv.Sobel(img, cv.CV_32F, 1, 0)
    gy = cv.Sobel(img, cv.CV_32F, 0, 1)
    # 计算梯度的幅值和方向
    mag, ang = cv.cartToPolar(gx, gy)
    # 将梯度方向量化到0到16之间的箱子
    bins = np.int32(bin_n * ang / (2 * np.pi))  # 将角度量化到0到16之间的箱子
    # 将图像分割成4个子图像 图像大小是20x20
    bin_cells = bins[:10, :10], bins[10:, :10], bins[:10, 10:], bins[10:, 10:]
    mag_cells = mag[:10, :10], mag[10:, :10], mag[:10, 10:], mag[10:, 10:]
   
    # 计算每个子图像的HOG特征
    # 解释语法: np.bincount(b.ravel(), m.ravel(), bin_n)
    # b.ravel()将b变成一维数组，m.ravel()将m变成一维数组，bin_n是箱子数
    # hists是一个列表，列表中的每个元素是一个64位的向量
    # hists[0]是第一个子图像的HOG特征，hists[1]是第二个子图像的HOG特征，以此类推
    # hists[0]的前16个元素是第一个子图像的第一个细胞单元的HOG特征
    # hists[0]的后16个元素是第一个子图像的第二个细胞单元的HOG特征
    hists = [np.bincount(b.ravel(), m.ravel(), bin_n) for b, m in zip(bin_cells, mag_cells)]
    # 将4个子图像的HOG特征进行连接
    hist = np.hstack(hists)  # hist是一个64位向量
    return hist

def main():
    
    img = cv.imread(cv.samples.findFile('../data/digits.png'), 0)
    if img is None:
        raise Exception("请将digits.png图像放置在samples/data目录中！")
    #print(img.shape) (1000, 2000)
    # 将图像分割成50x100个小图像 每个小图像的大小是20x20 
    # np.vsplit(img, 50)将图像分割成50个小图像，每个小图像的大小是2000x20
    cells = [np.hsplit(row, 100) for row in np.vsplit(img, 50)]
    # print(cells[0][0].shape) # 50x100 (20, 20)
    # 前半部分是训练数据，后半部分是测试数据
    # 这里遍历cells的时候，cells是一个50x100的列表,所以他遍历的是一个100x20x20的列表
    train_cells = [i[:50] for i in cells] #50x50 (20, 20) 
    test_cells = [i[50:] for i in cells]  #50x50 (20, 20)

    # 对训练图像进行去倾斜处理
    #  [list(map(deskew, row))是对每一行的图像进行去倾斜处理
    deskewed = [list(map(deskew, row)) for row in train_cells]
    # 计算去倾斜图像的HOG特征
    hogdata = [list(map(hog, row)) for row in deskewed]
    # 将HOG特征转换成numpy的数组 64是因为HOG特征是64位的向量
    trainData = np.float32(hogdata).reshape(-1, 64)
    responses = np.repeat(np.arange(10), 250)[:, np.newaxis]

    # 创建一个SVM模型并设置参数
    svm = cv.ml.SVM_create()
    svm.setKernel(cv.ml.SVM_LINEAR)  # 线性核函数
    svm.setType(cv.ml.SVM_C_SVC)     # C-SVC分类器
    svm.setC(2.67)                   # C值
    svm.setGamma(5.383)              # gamma值
    # 训练SVM模型
    svm.train(trainData, cv.ml.ROW_SAMPLE, responses)
    svm.save('svm_data.dat')

    # 对测试图像进行去倾斜处理
    deskewed = [list(map(deskew, row)) for row in test_cells]
    # 计算去倾斜图像的HOG特征
    hogdata = [list(map(hog, row)) for row in deskewed]
    testData = np.float32(hogdata).reshape(-1, bin_n * 4)
    # 使用SVM模型进行预测
    result = svm.predict(testData)[1]
    mask = result == responses
    correct = np.count_nonzero(mask)
    accuracy = correct * 100.0 / result.size
    print("准确率: {:.2f}%".format(accuracy))

if __name__ == '__main__':
    main()


准确率: 93.80%


In [14]:
import cv2
import numpy as np

# 读取输入图像
image = cv2.imread('../data/HappyFish.jpg', 0)

# 对图像进行二值化处理
_, binary_image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)

# 计算图像的几何矩
moments = cv2.moments(binary_image)

# 获取某些特定的矩的值
m00 = moments['m00']
m10 = moments['m10']
m01 = moments['m01']

# 打印矩的值
print("m00:", m00)
print("m10:", m10)
print("m01:", m01)


m00: 10240545.0
m10: 1217487555.0
m01: 978796335.0
